![Alt Text](generative_smiles.png)

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
import itertools

def calculate_rule_of_five_violations(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        return None  # Invalid SMILES

    mw = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    h_donors = Descriptors.NumHDonors(mol)
    h_acceptors = Descriptors.NumHAcceptors(mol)
    
    violations = 0
    if mw > 500: violations += 1
    if logp > 5: violations += 1
    if h_donors > 5: violations += 1
    if h_acceptors > 10: violations += 1
    
    return violations

def generate_SMILES(original_smiles, core_smiles):
    replacements = ["C", "N", "O"]
    c_positions = [i for i, char in enumerate(original_smiles) if char == "C"]
    combinations = itertools.product(replacements, repeat=len(c_positions))

    cyclic_smiles_list_with_br = []
    for combo in combinations:
        modified_smiles = list(original_smiles)
        for pos, replacement in zip(c_positions, combo):
            modified_smiles[pos] = replacement
        cyclic_smiles_list_with_br.append("".join(modified_smiles) + "Br")

    core_mol = Chem.MolFromSmiles(core_smiles)

    new_smiles_list = []

    for group_smiles in cyclic_smiles_list_with_br:
        group_mol = Chem.MolFromSmiles(group_smiles)

        if core_mol is None or group_mol is None:
            continue

        rxn = AllChem.ReactionFromSmarts("[*:1]Br.[Br][*:2]>>[*:1][*:2]")

        product_sets = rxn.RunReactants((core_mol, group_mol))

        if product_sets and len(product_sets[0]) > 0:
            product = product_sets[0][0] if product_sets[0] else None
            if product:
                new_smiles = Chem.MolToSmiles(product)
                new_smiles_list.append(new_smiles)

    # Filter based on Rule of Five violations
    filtered_smiles_list = []
    for smiles in new_smiles_list:
        violations = calculate_rule_of_five_violations(smiles)
        if violations is not None and violations < 4:
            filtered_smiles_list.append(smiles)

    for i, smiles in enumerate(filtered_smiles_list):
        print(f"{i+1}: {smiles} (Violations: {calculate_rule_of_five_violations(smiles)})")

    return filtered_smiles_list

inputs = ["CC1CCCC(C)C1", "CCC1CCCCC1C", "CC1=C(C)C=CC1",
          "CC1=CC(C)=CC1", "CC1=CC=C(C)C1", "CCC1CCCCC1",
          "CC1CCCCC1", "CC1CCCC1", "O1C=CC=C1", "CC1=COC=C1",
          "C1=CC=CC=C1", "CC1=CC=CC=C1", "C1CCC1",
          "CC1CCC1", "C1=CC=C1", "CC1=CC=C1"]

for input in inputs:
    generate_SMILES(input, "OC(O)C1=CC=NC=C1NCC1CCCC2=C1C=CC(NBr)=C2")
for input in inputs:
    generate_SMILES(input, "OC(O)C1=CC=NC=C1NCC1CCCC2=C1C=CC(N(C)Br)=C2")